In [1]:
import cv2
import numpy as np
import datetime
import gdown
import os

In [2]:
def calculate_timestamp(frame_number, fps):
    return frame_number / fps

In [3]:
def detect_ball_color(frame, lower_color, upper_color):
    mask = cv2.inRange(frame, lower_color, upper_color)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        contour = max(contours, key=cv2.contourArea)
        (x, y), _ = cv2.minEnclosingCircle(contour)
        return int(x), int(y)

    return None

In [4]:
# fps = cap.get(cv2.CAP_PROP_FPS)
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [6]:
def process_video(video_path):
    color_ranges = {
        'red': ([0, 0, 100], [50, 50, 255]),
        'blue': ([100, 0, 0], [255, 50, 50]),
        'green': ([0, 100, 0], [50, 255, 50]),
    }

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter('processed_video.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

    ball_positions = {color: None for color in color_ranges}
    previous_quadrant = {color: None for color in color_ranges}

    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        for color, (lower, upper) in color_ranges.items():
            ball_positions[color] = detect_ball_color(frame, np.array(lower), np.array(upper))

        for color, position in ball_positions.items():
            if position:
                x, y = position
                quadrant_number = 1 + (x // (frame_width // 2)) + 2 * (y // (frame_height // 2))
                quadrant_number = min(max(quadrant_number, 1), 4)

                if previous_quadrant[color] is not None and previous_quadrant[color] != quadrant_number:
                    timestamp = calculate_timestamp(frame_number, fps)
                    event_type = 'Entry' if quadrant_number > previous_quadrant[color] else 'Exit'

                    cv2.putText(frame, f"{event_type} {timestamp:.2f}s", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    with open('event_data.txt', 'a') as f:
                        f.write(f"{timestamp:.2f}s, {quadrant_number}, {color}, {event_type}\n")
                        print(f"{timestamp:.2f}s, {quadrant_number}, {color}, {event_type}")

                previous_quadrant[color] = quadrant_number

        for color, position in ball_positions.items():
            if position:
                x, y = position
                cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)

        out.write(frame)

        cv2.imshow('Processed Video', frame)
        if cv2.waitKey(30) & 0xFF == 27:
            break

        frame_number += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [8]:
if __name__ == "__main__":
    video_path = r'AIAssignmentvideo.mp4'
    open(r'event_data.txt', 'w').close()
    process_video(video_path)

0.47s, 2, blue, Exit
0.57s, 4, blue, Entry
0.67s, 3, red, Exit
0.70s, 4, red, Entry
0.77s, 3, red, Exit
0.80s, 4, red, Entry
0.83s, 3, red, Exit
0.87s, 4, red, Entry
1.00s, 3, red, Exit
1.03s, 4, red, Entry
1.07s, 2, blue, Exit
1.10s, 3, red, Exit
1.10s, 4, blue, Entry
1.13s, 4, red, Entry
1.13s, 2, blue, Exit
1.17s, 4, blue, Entry
1.23s, 3, red, Exit
1.23s, 2, blue, Exit
1.27s, 4, blue, Entry
1.33s, 4, red, Entry
1.43s, 3, red, Exit
1.53s, 2, blue, Exit
1.57s, 4, red, Entry
1.57s, 4, blue, Entry
1.60s, 3, red, Exit
1.70s, 4, red, Entry
1.73s, 3, red, Exit
1.73s, 2, blue, Exit
1.80s, 4, blue, Entry
1.83s, 2, blue, Exit
1.87s, 4, blue, Entry
1.90s, 4, red, Entry
1.97s, 3, red, Exit
2.03s, 4, red, Entry
2.13s, 3, red, Exit
2.17s, 2, blue, Exit
2.20s, 4, red, Entry
2.20s, 4, blue, Entry
2.23s, 2, blue, Exit
2.50s, 4, blue, Entry
3.03s, 3, red, Exit
3.07s, 4, red, Entry
3.10s, 3, red, Exit
3.33s, 4, red, Entry
3.40s, 3, red, Exit
3.43s, 4, red, Entry
3.53s, 3, red, Exit
3.57s, 4, red, Entr